In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
# import detectron2
# from detectron2.utils.logger import setup_logger

# setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

Generate single cells from a mask dataset and write single cell json data to disk

In [ ]:
pos = "XY1"

In [ ]:

# dataset_dir_path = Path(
#     f"../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/{pos}/"
# )

# mask_dataset_path = Path(f"../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/{pos}/seg")

# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
# time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
# time2url = {i: path for i, path in enumerate(time2url)}
# dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

# from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

# scs_write_path = f"./datasets/test_scs_EBSS_starvation/{pos}/scs.json"
# dataset_dir = Path(f"./datasets/test_scs_EBSS_starvation/{pos}/datasets")
# SingleCellStatic.write_single_cells_json(single_cells, scs_write_path, dataset_dir=dataset_dir)


Read single cell object data

In [ ]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection, SingleCellStatic
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)

# all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
all_scs_json_path = f"./datasets/test_scs_EBSS_starvation/{pos}/scs.json"
all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

In [ ]:
dic_dataset = all_scs[0].img_dataset

read trained an MMAction model


In [ ]:
import torch
from pathlib import Path
model_dir = Path(r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2/")
# path = r"./scripts/mmdetection_classify/work_dirs/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v6-combined-clipL=2/best_acc_top1_epoch_28.pth"
# model = torch.load(path)

In [ ]:
from mmengine.config import Config, DictAction
from mmaction.registry import MODELS
config_file = str(model_dir / "tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py")
checkpoint_file = str(model_dir / "best_acc_top1_epoch_28.pth")
cfg = Config.fromfile(config_file)

In [ ]:
# model = MODELS.build()
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer
model = init_recognizer(config_file, checkpoint_file, device='cuda:0')

In [ ]:
from mmcv.video.io import VideoReader
import tempfile
import numpy as np

In [ ]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

In [ ]:
sample = sctc.get_all_trajectories()[11].get_all_scs()

def gen_sc_frames(sample):
    print("length of sample:", len(sample))
    frames, frame_masks = video_frames_and_masks_from_sample(sample)
    combined_frames = combine_video_frames_and_masks(frames, frame_masks)
    sct_input_frames = [np.array(frame) for frame in combined_frames]
    return sct_input_frames


In [ ]:
sct_input_frames = gen_sc_frames(sample)

In [ ]:
import cv2
import os
import uuid
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\mitosis_1_mask.mp4"
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\normal_6.mp4"
# input_frames = sct_input_frames[:5]

def predict_on_frame_windows(model, frames, window_size=5, step_size=1, fps = 1):
    frame_windows = []
    for i in range(0, len(frames), step_size):
        if i + window_size > len(frames):
            break
        frame_window = frames[i:i+window_size]
        frame_windows.append(frame_window)
    results = []
    for frame_window in frame_windows:
        results.append(predict_on_frames(model, frame_window, fps))
    return results

def predict_on_frames(model, input_frames, fps, tmp_dir = "./tmp_video"):
    os.makedirs(tmp_dir, exist_ok=True)
    tmp_id = uuid.uuid4()
    input_tmp_filename = os.path.join(tmp_dir, f"tmp-{tmp_id}.mp4")

    def save_frames(frames, filename):
        with open(filename, "wb+") as tmp_video_file:
            # mmcv.frames2video(frames, tmp_video_file.name, fps=1)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            # Create a VideoWriter object to write the frames to the output file
            height, width, channels = frames[0].shape
            video_writer = cv2.VideoWriter(tmp_video_file.name, fourcc, fps, (width, height))

            # Write the frames to the output file
            for frame in frames:
                video_writer.write(frame)
            video_writer.release()
            tmp_video_file.flush()
            return tmp_video_file
        
    tmp_video_file = save_frames(input_frames, input_tmp_filename)
    results = inference_recognizer(model, tmp_video_file.name)
    predicted_label = results.pred_labels.item.cpu().numpy()[0]
    new_filename = os.path.join(tmp_dir, f"label-{predicted_label}-tmp-{tmp_id}.mp4")

    # convert the first channel in input_frames to RGB and save the video
    input_frames = np.array(input_frames)
    input_frames_rgb = [cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB) for frame in input_frames[..., 0]]
    raw_frame_video_filename = os.path.join(tmp_dir, f"label-{predicted_label}-tmp-{tmp_id}-raw.mp4")
    save_frames(input_frames_rgb, raw_frame_video_filename)

    # move file to new filename
    os.rename(input_tmp_filename, new_filename)
    return predicted_label

In [ ]:
# results = predict_on_frame_windows(model, sct_input_frames, window_size=20, step_size=1)

In [ ]:
# predict on sctc samples
from tqdm import tqdm
def predict_on_sctc_samples(model, sctc, window_size=5, step_size=1, fps = 1):
    all_samples_results = {}
    for track_id, trajectory in tqdm(sctc):
        sample = trajectory.get_all_scs()
        sct_input_frames = gen_sc_frames(sample)
        sample_results = predict_on_frame_windows(model, sct_input_frames, window_size=window_size, step_size=step_size, fps=fps)
        if 0 in sample_results:
            print("track_id:", track_id, "contains mitosis")
        all_samples_results[track_id] = sample_results
    return all_samples_results

In [ ]:
!rm -r ./tmp_video

In [ ]:
track_id_to_sample_results = predict_on_sctc_samples(model, sctc, window_size=10, step_size=5, fps=1)

Visualize mitosis cells

In [ ]:
from livecell_tracker.core.sct_operator import create_scts_operator_viewer

# mitosis_tids = [7, 4, 1, 16] # EBSS less than 24h, xy1
mitosis_tids = [tid for tid in track_id_to_sample_results if 0 in track_id_to_sample_results[tid]]
mitosis_tids

In [ ]:
from livecell_tracker.core.sct_operator import create_scts_operator_viewer, create_scs_edit_viewer

# scts_operator = create_scs_edit_viewer(single_cells, img_dataset = dic_dataset, time_span=(145, 155))
# # If you would like to start from sctc, you can use the following code
# scts_operator = create_scts_operator_viewer(sctc, img_dataset = dic_dataset) #, time_span=(1, 2))
# scts_operator = create_scts_operator_viewer(sctc.subset(mitosis_tids), img_dataset = dic_dataset) #, time_span=(1, 2))

In [ ]:
# results = inference_recognizer(model, r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data_v5\videos\mitosis_2_combined_padding-40.mp4")
# results

In [ ]:
import pandas as pd
video_dir = r"./notebook_results/mmaction_train_data_v5/videos"

mmaction_data_tsv = r"./notebook_results/mmaction_train_data_v6/test_data_combined.txt"

# df columns are path and label
mmaction_data_df = pd.read_csv(mmaction_data_tsv, sep=" ", header=None)
mmaction_data_df.columns = ["path", "label"]
mmaction_data_df[:2]

In [ ]:
from tqdm import tqdm
# test on all video in the df
for i, row in tqdm(mmaction_data_df.iterrows()):
    video_path = os.path.join(video_dir, row["path"])
    results = inference_recognizer(model, video_path)
    predicted_label = results.pred_labels.item.cpu().numpy()[0]
    if predicted_label != row["label"]:
        print("wrong prediction:", row["path"])